# Web Scraping of Data for Rating Review Prediction Project

### Data Collection Phase 
You have to scrape at least 20000 rows of data. You can scrape more data as well, it’s up to you. 
more the data better the model

In this section you need to scrape the reviews of different laptops, Phones, Headphones, smart 
watches, Professional Cameras, Printers, Monitors, Home theater, Router from different ecommerce websites.

Basically, we need these columns
1) reviews of the product.
2) rating of the product.

You can fetch other data as well, if you think data can be useful or can help in the project. It 
completely depends on your imagination or assumption.

In [1]:
#importing the required libraries
import pandas as pd
import numpy as np
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException, ElementNotVisibleException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import warnings
warnings.filterwarnings('ignore')

In [2]:
# getting the template
def get_template(search_term,page_no=1):
    """Generate a URL for search Term with page number"""
    search_term = search_term.replace(' ','+')
    template = f'https://www.amazon.in/s?k={search_term}&page={page_no}&qid=1623864210&ref=sr_pg_{page_no}'
    
    return template

In [3]:
#getting the URL
def get_url():#the function that generated URL with the search tearm.
    iteams = ['laptops', 'Phones', 'Headphones', 'smart watches', 'Professional Cameras', 'Printers', 'scanners', 'keyboard', 'monitors', 'Home theater', 'router']
    URL = []     
    for i in iteams:
        for j in range(1,15):
            URL.append(get_template(i,j))
    return URL  

In [4]:
#getting the review URL
def get_review_url(): #the function that scraps the URL of the front page of the reviews and rattings with the search tearm.
    
    hreff = []
    driver = webdriver.Chrome("D:\Downloads\chromedriver_win\chromedriver.exe") 
    driver.maximize_window()

    URL = get_url()
    for k in URL:
        driver.get(k) #Opening with the URL template

        for l in driver.find_elements(By.XPATH,"//a[@class = 'a-link-normal']"):
            hreff.append(l.get_attribute("href"))   
    return hreff

In [5]:
href = get_review_url()
len(href)

1616

In [6]:
def get_review(): #the function that scraps the URL of all the reviews and rattings
    
    driver = webdriver.Chrome(r"D:\Downloads\chromedriver_win\chromedriver.exe") #Calling the Web Driver
    driver.maximize_window()
    ratings = []
    review = []

    for i in href:
        driver.get(i) #Opening with the URL template
        
        try: #scraping the URL of the full review pages
            link = driver.find_element(By.XPATH,"//a[@data-hook='see-all-reviews-link-foot']")
            link = link.get_attribute("href")
            link= link.split('ref=cm')[0]
        except:
            pass     
        
        for i in range(1,5): #Scraping Reviews and Rattings
            l1= f'ref=cm_cr_getr_d_paging_btm_next_{i}?ie=UTF8&reviewerType=all_reviews&pageNumber={i}'
            l = link+ l1 
            
            
            driver.get(l)
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            results = soup.find_all('div',{'class' : 'a-section celwidget'})
        
            for item in results:
                    try:
                        rev = item.find('span', {'data-hook': "review-body"})
                        atag = item.find(['a'], class_ = "a-link-normal")
                        if rev.span.text.replace('\n','').strip() and atag['title'][0]:
                            review.append(rev.span.text.replace('\n','').strip())
                            ratings.append(atag['title'][0])
                        else:
                            break
                    except:
                        break
       
    driver.close()
    
    rating = pd.DataFrame({'Rattings':ratings, "Review": review})
    
    #saving the dataframe in csv
    rating.to_csv("Rattings1.csv",index=False) #Creating CSV
    
    return rating

In [7]:
Rat = get_review()
Rat

,Rattings,Review
0,1,Please don't purchase any Lenovo product.4 mon...
1,1,Please don't purchase any Lenovo product.4 mon...
2,1,Please don't purchase any Lenovo product.4 mon...
3,4,the laptop whole package is awesome. although ...
4,4,"Pros:Good specs, others have written enough ab..."
...,...,...
42075,5,I have several of thes across the whole house....
42076,2,This Deco x60 has many limitations compared to...
42077,4,HiAlways had a problem of getting strong enoug...
42078,4,I was originally using deco E 4 route and was ...


In [8]:
Rat.to_csv("Ratings_scraped.csv",index=False)# saving Data frame as CSV